# Climate Change

Data source [Kaggle](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data?select=GlobalLandTemperaturesByState.csv)

Global Land and Ocean-and-Land Temperatures (GlobalTemperatures.csv):

- `Date`: starts in 1750 for average land temperature and 1850 for max and min land temperatures and global ocean and land temperatures
- `LandAverageTemperature`: global average land temperature in celsius
- `LandAverageTemperatureUncertainty`: the 95% confidence interval around the average
- `LandMaxTemperature`: global average maximum land temperature in celsius
- `LandMaxTemperatureUncertainty`: the 95% confidence interval around the maximum land temperature
- `LandMinTemperature:` global average minimum land temperature in celsius
- `LandMinTemperatureUncertainty`: the 95% confidence interval around the minimum land temperature
- `LandAndOceanAverageTemperature`: global average land and ocean temperature in celsius
- `LandAndOceanAverageTemperatureUncertainty`: the 95% confidence interval around the global average land and ocean temperature


The raw data comes from the [Berkeley Earth data page](https://berkeleyearth.org/data/).

In [2]:
import pandas as pd
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np 

In [3]:
temps = pd.read_csv("climate_data.csv")
temps.head(10)

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1980-01-01,2.956,0.087,8.468,0.080,-2.457,0.124,13.812,0.049
1,1980-02-01,3.652,0.099,9.260,0.129,-1.864,0.115,14.112,0.050
2,1980-03-01,5.367,0.103,11.240,0.160,-0.379,0.126,14.646,0.052
3,1980-04-01,8.935,0.106,14.821,0.129,3.109,0.175,15.578,0.052
4,1980-05-01,11.770,0.089,17.713,0.234,6.131,0.172,16.314,0.050
5,1980-06-01,13.582,0.134,19.375,0.185,7.901,0.299,16.806,0.058
6,1980-07-01,14.742,0.144,20.466,0.179,9.102,0.214,17.171,0.060
7,1980-08-01,14.189,0.221,19.814,0.117,8.563,0.089,17.049,0.077
8,1980-09-01,12.321,0.080,18.121,0.152,6.642,0.146,16.427,0.051
9,1980-10-01,9.550,0.064,15.170,0.118,4.051,0.112,15.450,0.048
